## 5-2. 그룹 연산 메소드(적용-결합 단계)

- 데이터 집계

앞에서 분할한 그룹 객체에 대하여 각 그룹별 평균을 계산한 것처럼, 그룹 객체에 다양한 연산을 적용할 수 있다.<br>
이 과정을 데이터 집계(aggregation)라고 부른다.<br><br>

집계 기능을 내장하고 있는 판다스 기본 함수에는<br>
`mean(), max(), min(), sum(), count(), size(), var(), std(), describe(), info(), first(), last()` 등이 있다.<br>
다음의 예제에서는 각 그룹의 표준편차를 집계하는 `std()` 메소드를 기준으로 설명한다.

- 표준편차 데이터 집계(내장 함수): `group 객체.std()`

먼저 각 그룹에 대하여 각 열의 표준편차를 계산하고, 각 그룹을 행 인덱스로 갖는 데이터프레임을 반환한다.<br>
요금을 나타내는 'fare' 열을 보면 1등석인 'First' 클래스의 요금의 표준편차가 2~3등석보다 훨씬 크다.<br>
한편 'fare' 열을 따로 지정하여 집계 연산을 적용하면 시리즈를 반환한다.


In [1]:
import pandas as pd
import seaborn as sns

# titanic 데이터셋에서 age, sex 등 5개 열을 선택하여 데이터프레임 만들기
titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age', 'sex', 'class', 'fare', 'survived']]

# class 열을 기준으로 분할
grouped = df.groupby(['class'])

# 각 그룹에 대한 모든 열의 표준편차를 집계하여 데이터프레임으로 변환
std_all = grouped.std()
print(std_all)
print('\n')
print(type(std_all))
print('\n')

# 각 그룹에 대한 fare 열의 표준편차를 집계하여 시리즈로 변환
std_fare = grouped.fare.std()
print(std_fare)
print('\n')
print(type(std_fare))


              age       fare  survived
class                                 
First   14.802856  78.380373  0.484026
Second  14.001077  13.417399  0.500623
Third   12.495398  11.778142  0.428949


<class 'pandas.core.frame.DataFrame'>


class
First     78.380373
Second    13.417399
Third     11.778142
Name: fare, dtype: float64


<class 'pandas.core.series.Series'>


---

집계 연산을 처리하는 사용자 정의 함수를 그룹 객체에 적용하려면 `agg()` 메소드를 사용한다.<br>
다음의 예제에서는 최대값과 최소값의 차를 계산하는 함수를 정의하여 사용한다.

- agg() 메소드 데이터 집계: `group 객체.agg(매핑 함수)`



In [3]:
# 그룹 객체에 agg() 메소드 적용 - 사용자 정의 함수를 인자로 전달
def min_max(x):  # 최대값 - 최소값
    return x.max() - x.min()


# 각 그룹의 최대값과 최소값의 차이를 계산하여 그룹별로 집계
agg_minmax = grouped.agg(min_max)
print(agg_minmax.head())


          age      fare  survived
class                            
First   79.08  512.3292         1
Second  69.33   73.5000         1
Third   73.58   69.5500         1


/var/folders/c7/9g12sj_s5pn6d36sm6m03b5w0000gn/T/ipykernel_2911/1099442612.py:7: FutureWarning: ['sex'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  agg_minmax = grouped.agg(min_max)


---

각 그룹별로 연산이 가능한 열에 대한 최대값과 최소값의 차를 구하면 데이터 값의 분포 범위를 알 수 있다.<br>
앞에서 표준 편차를 구하는 것과 비슷한 해석이 가능하다.<br>

동시에 여러 개의 함수를 사용하여 각 그룹별 데이터에 대한 집계 연산을 처리할 수 있다.<br>
각각의 열에 여러 개의 함수를 일괄 적용할 때는 리스트 형태로 인수를 전달하고,<br>
열마다 다른 종류의 함수를 적용하려면 {열 : 함수} 형태의 딕셔너리를 전달한다.

- 모든 열에 여러 함수를 매핑: `group 객체.agg([함수1, 함수2, 함수3 ...])`
- 각 열마다 다른 함수를 매핑: `group 객체.agg({'열1': 함수1, '열2': 함수2 ...})`

예제에서 agg_all 변수는 grouped 객체의 각 열에 2개의 함수('min', 'max')를 일괄 적용하여 그룹별로 집계한 결과이다.<br>
한편 agg_sep 변수는 'fare' 열에는 2개 함수('min', 'max')를 적용하고, 'age' 열에는 다른 종류의 함수('mean')를 적용하여 집계한 결과이다.



In [5]:
# 여러 함수를 각 열에 동일하게 적용하여 집계
agg_all = grouped.agg(['min', 'max'])
print(agg_all.head())
print('\n')

# 각 열마다 다른 함수를 적용하여 집계
agg_sep = grouped.agg({'fare': ['min', 'max'], 'age': 'mean'})
print(agg_sep.head())


         age           sex       fare           survived    
         min   max     min   max  min       max      min max
class                                                       
First   0.92  80.0  female  male  0.0  512.3292        0   1
Second  0.67  70.0  female  male  0.0   73.5000        0   1
Third   0.42  74.0  female  male  0.0   69.5500        0   1


       fare                  age
        min       max       mean
class                           
First   0.0  512.3292  38.233441
Second  0.0   73.5000  29.877630
Third   0.0   69.5500  25.140620


앞에서 2개의 함수를 리스트 형태로 입력하면 각 열에 대하여 2개 함수의 연산 결과를 각각 집계하여 다른 열로 구분하여 표시한다.<br>
이때 함수명을 열 이름에 추가하여 2중 열 구조를 만든다.
